In [18]:
from joblib import load

vectorizer = load("../models/lai_ms_tfidf_vectorizer.joblib")
model, metadata = load("../models/lai_ms_model_classifier.joblib")

vectorizer, model, metadata

(TfidfVectorizer(max_df=0.8, max_features=1000, min_df=5, ngram_range=(1, 2)),
 CalibratedClassifierCV(estimator=LinearSVC(class_weight='balanced',
                                            random_state=271828)),
 {'xgboost': '3.1.2', 'sklearn': '1.8.0', 'numpy': '2.3.5'})

In [19]:
X_new = vectorizer.transform(["Gostaria acessar dados ministério"])
print("NNZ:", X_new.nnz)
print("Shape:", X_new.shape)

NNZ: 2
Shape: (1, 1000)


In [3]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 26.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [4]:
import spacy

nlp = spacy.load('pt_core_news_sm')
stopwords_spacy = nlp.Defaults.stop_words
list(stopwords_spacy)[:10]

['apoia',
 'vinte',
 'oito',
 'põe',
 'tempo',
 'como',
 'próxima',
 'comprida',
 'cedo',
 'talvez']

In [5]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords_nltk = stopwords.words('portuguese')
list(stopwords_nltk)[:10]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/isaaclourenco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as']

In [6]:
both_stopwords = set(stopwords_nltk) | set(stopwords_spacy)

In [7]:
import re

def remove_stop_word(text):
    text = re.sub(r'[^\w\s]', '', text)

    tokens = text.split()

    tokens = filter(lambda token: token not in both_stopwords, tokens)

    return ' '.join(tokens)

In [8]:
def spacy_lemmatizer(text):
    doc = nlp(text)

    txt = [token.lemma_ for token in doc]

    txt = [word for word in txt if len(word) > 2]

    return ' '.join(txt)

In [9]:
import pandas as pd

def predict(text, vectorizer, model):
    texts = pd.Series([text])
    no_stop_word_texts = texts.str.lower().apply(remove_stop_word)
    lemmatized_texts = no_stop_word_texts.str.lower().apply(spacy_lemmatizer)
    X_new = vectorizer.transform(lemmatized_texts)
    return X_new.nnz, model.predict_proba(X_new)

In [ ]:
lais_outros = [
    "Eu tenho direito de acessar meus dados armazenados no sistema da justiça.",
    "Processo de número 500 me dá direito de acessar os dados gerados.",
    "Tenho meus direitos. Quero que vocês me enviem um relatório contendo os últimos processos judiciais no meu nome.",
    "solicitar acessar dados justiça",
    "solicitar acessar dados ministério justiça",
]

In [30]:
for lai in lais_outros:
    print(lai)
    nnz, pred_proba = predict(lai, vectorizer=vectorizer, model=model)
    print("NNZ:", nnz)
    print("Outro:", pred_proba[0][0])
    print("MS:", pred_proba[0][1])
    print()

Gostaria de solicitar meu prontuário médico, por gentiliza.
NNZ: 4
Outro: 0.5541690036284468
MS: 0.4458309963715532

Eu tenho direito de acessar meus dados armazenados no sistema da justiça.
NNZ: 4
Outro: 0.9981231350720311
MS: 0.0018768649279689333

Processo de número 500 me dá direito de acessar os dados gerados.
NNZ: 5
Outro: 0.9707633890474436
MS: 0.029236610952556353

Tenho meus direitos. Quero que vocês me enviem um relatório contendo os últimos processos judiciais no meu nome.
NNZ: 7
Outro: 0.971775361865998
MS: 0.028224638134001844

solicitar acessar dados justiça
NNZ: 4
Outro: 0.9963477540930519
MS: 0.003652245906948186

solicitar acessar dados ministério justiça
NNZ: 5
Outro: 0.9894028625836405
MS: 0.010597137416359325



In [39]:
lais_ms = [
    "Gostaria de solicitar meu prontuário médico, por gentiliza.",
    "Quero o prontuário gerado pelo médico Alberto Carlos que estava em plantão no dia 14. Ele me atendeu.",
    "Quero acesso aos medicamentos mais demandados pela polução no ano de 2024.",
    "Quero um relatório contendo as vacinas mais aplicadas no ano de 2024 em Santa Cruz, Rio Grande do Norte.",
    "Preciso de um relatório detalhado da frequências dos médicos em Unidades de Pronto Atendimento em Natal no último ano.",
]

In [40]:
for lai in lais_ms:
    print(lai)
    nnz, pred_proba = predict(lai, vectorizer=vectorizer, model=model)
    print("NNZ:", nnz)
    print("Outro:", pred_proba[0][0])
    print("MS:", pred_proba[0][1])
    print()

Gostaria de solicitar meu prontuário médico, por gentiliza.
NNZ: 4
Outro: 0.5541690036284468
MS: 0.4458309963715532

Quero o prontuário gerado pelo médico Alberto Carlos que estava em plantão no dia 14. Ele me atendeu.
NNZ: 4
Outro: 0.7676288566896332
MS: 0.2323711433103668

Quero acesso aos medicamentos mais demandados pela polução no ano de 2024.
NNZ: 4
Outro: 0.4624400085510606
MS: 0.5375599914489394

Quero um relatório contendo as vacinas mais aplicadas no ano de 2024 em Santa Cruz, Rio Grande do Norte.
NNZ: 6
Outro: 0.9576437417926027
MS: 0.0423562582073974

Preciso de um relatório detalhado da frequências dos médicos em Unidades de Pronto Atendimento em Natal no último ano.
NNZ: 7
Outro: 0.6111677831105864
MS: 0.38883221688941366



In [ ]:
lai = input()
nnz, pred_proba = predict(lai, vectorizer=vectorizer, model=model)
print("NNZ:", nnz)
print("Outro:", pred_proba[0][0])
print("MS:", pred_proba[0][1])
print()

NNZ: 3
Outro: 0.9796624097788731
MS: 0.02033759022112683

